In [29]:
import numpy as np
import random 
from collections import defaultdict

In [30]:
# return a random speed between 0 and 6 blocks per time step
def random_speed():
    return random.choice([0,1,2,3,4,5,6])

In [31]:
def random_init_cars(road_len, P_init,
                     speed_random=False, Vmax=6):
    cars = defaultdict(int)
    # discrete possible set of speeds
    speeds = [0,1,2,3,4,5,6]
    # zero indicates there is no car or a non-rear part of a car
    # is present
    road = np.array([0]*road_len)
    if speed_random==True:
        x=0
        i=1
        while x < road_len:
            if np.random.rand() <= P_init:
                # choose random speed uniformly in {0,6} range
                cars[i] = random_speed()
                road[x] = i
                x+=2
                i+=1
                continue
            x+=1
    else:
        x=0
        i=1
        while x < road_len:
            if np.random.rand() <= P_init:
                # if not random, start all cars at Vmax 
                cars[i]=Vmax
                road[x]=i
                x+=2
                i+=1
                continue
            x+=1
    return cars, road
cars, road = random_init_cars(road_len=int(50),
                              P_init=.5, speed_random=True)
print(cars, road)

defaultdict(<class 'int'>, {1: 0, 2: 5, 3: 3, 4: 2, 5: 1, 6: 6, 7: 4, 8: 2, 9: 6, 10: 3, 11: 2, 12: 1, 13: 4, 14: 2, 15: 2, 16: 6, 17: 2, 18: 2}) [ 0  0  0  1  0  0  2  0  3  0  4  0  0  0  0  0  0  0  0  5  0  0  6  0
  7  0  8  0  9  0 10  0 11  0  0  0 12  0 13  0  0 14  0 15  0 16  0 17
  0 18]


In [32]:
# get the indexes of all cars on the road array
def car_positions(road):
    return np.nonzero(road)[0]

car_possies = car_positions(road)
print(car_pos)

[ 3 12 14 21 24 35]


In [33]:
# Generate a new car randomly if the 
# first position of the road is free. 

# always returns the car and road array
def generate_new_cars(cars, road, p_gen=1,
                      speed_random=False, Vmax=6):
    if not road[0:2].any() and np.random.rand() <= p_gen:
        # give the new key a new label
        new_car = max(cars.keys())+1
        road[0] = new_car
        print('new car ', new_car)
        # if speed randomization is false, set speed to Vmax
        if speed_random==False:
            cars[new_car] = Vmax
        else:
            # choose random speed uniformly
            cars[new_car] = random_speed()
            print('random speed')
    return cars, road      

generate_new_cars(cars, road, speed_random=True)

new car  19
random speed


(defaultdict(int,
             {1: 0,
              2: 5,
              3: 3,
              4: 2,
              5: 1,
              6: 6,
              7: 4,
              8: 2,
              9: 6,
              10: 3,
              11: 2,
              12: 1,
              13: 4,
              14: 2,
              15: 2,
              16: 6,
              17: 2,
              18: 2,
              19: 3}),
 array([19,  0,  0,  1,  0,  0,  2,  0,  3,  0,  4,  0,  0,  0,  0,  0,  0,
         0,  0,  5,  0,  0,  6,  0,  7,  0,  8,  0,  9,  0, 10,  0, 11,  0,
         0,  0, 12,  0, 13,  0,  0, 14,  0, 15,  0, 16,  0, 17,  0, 18]))

In [34]:
# the speed is a function of the gap size
# simplest model: accelerate to speed = gap_size
def speed_update_one_car(car_pi, car_positions,
                 cars, road, Vmax=6):
    distance = car_positions[car_pi+1] - car_positions[car_pi] - 2
    i = road[car_positions[car_pi]]
    if distance >= Vmax:
        cars[i] = Vmax
    else:
        cars[i] = distance
    return cars

In [50]:
# update position of a single car given:
# 1) it's index into the car positions array    
# 2) the car positions array, array of indexes that say's where each car is on the road
#     the array is ordered such that consecutive elements correspond with consecutive cars.
#     F.e. element 0 can correspond with car 1, element 1 with car 2 and so on.
# 3) the cars dictionairy
# 4) road array

# 
def position_update_one_car(car_pi, car_posses,
                    cars, road):
    car_pos = car_posses[car_pi]
    i = road[car_pos]
    speed = cars[i]
    # if speed is zero, car doesn't move
    if speed==0:
        return road
    print('speed=', speed)
    print('car ', i)
    print('road segment=', 
          road[car_pos:car_pos+speed])
    neighbors = car_positions(road[car_pos:car_pos+speed])
    print('neighbor indices=',
          neighbors)
    # consider edge case when the updating the last car in the array,
    # it can dissapear off the edge when position+speed > len(array)
    if car_pi == len(car_posses)-1:
        # delete car if it disappears of the road
        if car_pos+speed > road.size-1:
            road[car_pos]=0
            del cars[i]
            # only return the updated car dict if a car is deleted
            return cars, road
        # otherwise, the car can get up to the exact last block
        else:
            road[car_pos+speed]=road[car_pos]
            road[car_pos]=0
            return road          
    # if not last car, update the position as normal
    # if there is only one neighbor (i.e. itself), the road 
    # is clear to move at it's current speed
    if neighbors.size==1:
        road[car_pos + speed] = road[car_pos]
        road[car_pos]=0
        return road
    # otherwise, move as close to the closest leader car 
    # as possible and adjust speed to this leader
    else:
        # dnn is relative distance to the first leader
        dnn = neighbors[1]
        print('distance to leader=',dnn)
        # if the relative distance is exactly 2, it means the 
        # car is already driving as close as it wants to,
        # and it can't move closer
        if dnn == 2:
            print('no change')
            return road
        # otherwise, the speed would result in a movement that 
        # is larger than the gap between itself and it's nearest
        # leader
        else:
            # minimum gap is defined as it's distance to the nearest leader
            # minus two, since there should always be one block between cars = 
            # distance of 2
            min_gap = dnn-2
            # if the speed is larger than this gap, is should simpy move 
            # as close as possible, i.e. this exact gap size, so it's
            # new position is within one block of it's nearest leader
            if speed > min_gap:
                road[car_pos + min_gap] = road[car_pos]
                road[car_pos]=0
                print('speed = min gap')
            # otherwise, there is a large enough gap given the current speed
            # to move and it simply updates the position using this speed
            else:
                road[car_pos + speed] = road[car_pos]
                road[car_pos]=0
                print('speed = speed')
        return road     

In [65]:
# update position of all cars 
def position_update(car_posses, cars, road):
    for cp in car_posses:
        print('CP ', cp)
        road = position_update_one_car(car_pi=cp, car_posses=car_posses,
                                        cars=cars, road=road)
        print(road)
    return road

In [72]:
cars, road = random_init_cars(road_len=100, P_init=0.3,
                     speed_random=False, Vmax=6)
print(cars)
possies = car_positions(road=road)
print(possies)
for cp in possies: 
    print(position_update_one_car(car_pi=cp, car_posses=possies,
                    cars=cars, road=road)
          )
#position_update(car_posses=possies, cars=cars, road=road)

defaultdict(<class 'int'>, {1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 6, 8: 6, 9: 6, 10: 6, 11: 6, 12: 6, 13: 6, 14: 6, 15: 6, 16: 6, 17: 6, 18: 6, 19: 6, 20: 6, 21: 6, 22: 6, 23: 6})
[ 4 10 12 16 19 23 25 30 33 40 42 44 56 59 61 66 68 71 82 87 94 97 99]
speed= 6
car  5
road segment= [5 0 0 0 6 0]
neighbor indices= [0 4]
distance to leader= 4
speed = min gap
[ 0  0  0  0  1  0  0  0  0  0  2  0  3  0  0  0  4  0  0  0  0  5  0  6
  0  7  0  0  0  0  8  0  0  9  0  0  0  0  0  0 10  0 11  0 12  0  0  0
  0  0  0  0  0  0  0  0 13  0  0 14  0 15  0  0  0  0 16  0 17  0  0 18
  0  0  0  0  0  0  0  0  0  0 19  0  0  0  0 20  0  0  0  0  0  0 21  0
  0 22  0 23]
speed= 6
car  11
road segment= [11  0 12  0  0  0]
neighbor indices= [0 2]
distance to leader= 2
no change
[ 0  0  0  0  1  0  0  0  0  0  2  0  3  0  0  0  4  0  0  0  0  5  0  6
  0  7  0  0  0  0  8  0  0  9  0  0  0  0  0  0 10  0 11  0 12  0  0  0
  0  0  0  0  0  0  0  0 13  0  0 14  0 15  0  0  0  0 16  0 17  0  0 18
  0  0  0 

IndexError: index 23 is out of bounds for axis 0 with size 23

In [ ]:
cars, road = random_init_cars(road_len=int(50),
                              P_init=0.3, speed_random=True)
print(road)
car_pos = car_positions(road)
lp = len(car_pos)-1
position_update(lp, car_pos, 
               cars, road)

In [ ]:
# MAIN EXPERIMENT LOOP
# The loop iterates for N times, each step applying the rules defined in the 
# update functions. Each update represents one time step.
# First a position update is made using the current time step + RT (reaction time)
# time steps, to represent the delayed reaction. The speed update is
# therefore always delayed by RT.

# Position update never crashes cars into one another,
# using the assumptions that cars can decelerate as fast as necessary.
# They never come closer than one block distance. If cars are at this minimum distance, 
# their next speed update sets their speed to 0, using the assumption that cars stop when 
# they get at this critical distance. Otherwise, the speed in blocks per time step 
# is exactly equal to the distance to it's leader, up to Vmax when there are more than 
# Vmax blocks free ahead. 
def main_loop(RT=0, N=10, 
              road_len, P_init, speed_random=False, 
              p_gen=0.3, Vmax=6):
    # initialize the road and cars 
    cars, road = random_init_cars(road_len=road_len, P_init=P_init,
                     speed_random=speed_random, Vmax=6)
    # make the initial positions array of all cars on the road
    car_posses = car_positions(road)
    #
    
    t_delay = 0
    for t in range(N):
        # the delay time incurred buy the reaction time, defines how long a car 
        # continues moving at the current speed
        t_delay = t + RT
        # update the position at the current velocity for t + RT time steps
        if t < t_delay:
            road = position_update(car_pi, car_posses,
                    cars, road)
        
    return cars, road

main_loop(road_len=100, P_init=0.3)

In [ ]:
#